### Method 1 : Getting results from the database through the GraphQL API

In [31]:
## importing libraries
import requests
import json
import pandas as pd
from flask import Flask, render_template

In [32]:
#defining the GraphQL query to get all employee nodes - with name and emp_id properties

query = """
query {
  employees {
    name
    emp_id
  }
}
"""

In [33]:
#submitting the query to our API and getting the data back
url = 'http://localhost:4000/'
r = requests.post(url, json={'query': query})
print(r.status_code)
print(r.text)

200
{"data":{"employees":[{"name":"Mick Doe","emp_id":101},{"name":"Srav","emp_id":109},{"name":"SRAV","emp_id":110},{"name":"Mick Doe","emp_id":103},{"name":"Sraf","emp_id":120},{"name":"Mick Doe","emp_id":104},{"name":"Mick Doe","emp_id":106},{"name":"Sravani","emp_id":107},{"name":"Sravani","emp_id":108}]}}



In [34]:
#converting from json format to python dictionary
json_data = json.loads(r.text)

In [35]:
#putting the data in a dataframe for better readability
df_data = json_data['data']['employees']
df = pd.DataFrame(df_data)

In [36]:
df

,name,emp_id
0,Mick Doe,101
1,Srav,109
2,SRAV,110
3,Mick Doe,103
4,Sraf,120
5,Mick Doe,104
6,Mick Doe,106
7,Sravani,107
8,Sravani,108


### Method 2 : Getting results directly from the database using the python driver

In [40]:
from neo4j import GraphDatabase

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password),encrypted=True)

    def close(self):
        self.driver.close()

    def get_all_employees(self):
        with self.driver.session() as session:
            result = session.read_transaction(self.show_all_employees)
            for row in result:
                print("Found employee: {name}, with Employee id: {eid}".format(name=row[0],eid=row[1]))

    @staticmethod
    def show_all_employees(tx):
        query = (
            "MATCH (e:Employee) "
            "RETURN e.name AS employee_name, e.emp_id as employee_id"
        )
        result = tx.run(query)
        return [row for row in result]


if __name__ == "__main__":
    uri = "bolt://98627160.databases.neo4j.io"
    user = "neo4j"
    password = "password"
    app = App(uri, user, password)
    app.get_all_employees()
    app.close()

Found employee: Mick Doe, with Employee id: 101
Found employee: Srav, with Employee id: 109
Found employee: SRAV, with Employee id: 110
Found employee: Mick Doe, with Employee id: 103
Found employee: Sraf, with Employee id: 120
Found employee: Mick Doe, with Employee id: 104
Found employee: Mick Doe, with Employee id: 106
Found employee: Sravani, with Employee id: 107
Found employee: Sravani, with Employee id: 108
